In [1]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import Dataset
import torchaudio
from torch.utils.data import DataLoader
import os
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [2]:
import flash
from flash.audio import SpeechRecognition, SpeechRecognitionData

2023-05-23 09:27:34.780291: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 09:27:34.826674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 09:27:35.483769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv("text/part2.csv")[:100]

In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, wav_dir):
        self.data = dataframe
        self.wav_dir = wav_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        file_id = self.data.iloc[index]['id']
        audio = os.path.join(self.wav_dir, f'{file_id}.wav')
        text = str(self.data.iloc[index]['form'])  

        max_len = len(text)
        text = text.ljust(max_len)  

        return audio, text

# 데이터셋 생성
wav_dir = './wav_all_stereo(fixed_length)'
dataset = CustomDataset(df, wav_dir)

In [13]:
len(dataset)

100

In [14]:
data = []
for i in range(len(dataset)):
    audio, text = dataset[i]
    data.append([audio, text])
df = pd.DataFrame(data, columns=['audio', 'text'])

In [29]:
df['audio'][0]

'./wav_all_stereo(fixed_length)/SDRW2000000749.1.1.1.wav'

In [15]:
df.to_csv('inference.csv',index=False,encoding='utf-8')

In [20]:
datamodule = SpeechRecognitionData.from_csv(
    'audio',
    test_file='inference.csv',
    batch_size=8  
)

In [21]:
model = SpeechRecognition.load_from_checkpoint("model_checkpoint.pt")

Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
trainer = flash.Trainer()

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
predictions = trainer.predict(model, datamodule=datamodule)

ValueError: An invalid dataloader was passed to `Trainer.predict(dataloaders=...)`. Either pass the dataloader to the `.predict()` method OR implement `def predict_dataloader(self):` in your LightningModule/LightningDataModule.

In [33]:
predictions = flash.Trainer.predict(['./wav_all_stereo(fixed_length)/SDRW2000000749.1.1.1.wav'])

AttributeError: 'list' object has no attribute 'accelerator'